# Analyzing borrowers’ risk of defaulting

Our project is to prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.


**the purposes of the project**:

The main purpose is finding out if a customer’s marital status and number of children have corelation with default on a loan, in order to decide wether to consider these data when building the **credit score** of a potential customer. The corelation of income level and of loan purpose with default rate will also be checked. Overall, the 4 questions we will answer will be the following:
1. Is there a connection between having kids and repaying a loan on time?
2. Is there a connection between marital status and repaying a loan on time?
3. Is there a connection between income level and repaying a loan on time?
4. How do different loan purposes affect on-time loan repayment?

## Openning and loading the data file 

In [1]:
# Loading library
import pandas as pd 

# Load the data
data = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

Now we will explore our data. We will see how many columns and rows it has, and look at a few rows to check for potential issues with the data.

In [2]:
# Let's see how many rows and columns our dataset has
data.shape  

(21525, 12)

In [3]:
# let's print the first 15 rows
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


We already can see in the printed data sample a few issues that will need further investigation and changes:
1. Missing values (for now seen only in 'income' and 'work experience' columns).
2. Values that will need to be consolidated to fewer unique values - in 'education' and 'purpose' columns.
3. Negative values in 'work experience' column.
4. The one positive value in 'work experience' column (in this sample) is an error (340266, i.e. more than 932 years...).
5. Values in 'income' and 'work experience' columns could be rounded (more convenient and doesn't seem critical).

We will look now into the general info to get the full picture of missing values and data types:

In [4]:
# Get info on data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Data types seem ok (later we will round the float values).
There are missing values only in 2 columns: 'days_employed' and 'total_income'. The number of missing values in these 2 columns is the same (which might hint that when one of these values is missing - the other is missing as well, which is logical since both relate to income). Let's see:  

In [5]:
# Let's look at the number of missing values in each column
data.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

The missing values seem symmetric between the 2 columns, but we will need to validate that. 

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
filtered_data = data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]  
filtered_data.shape

(2174, 12)

**Intermediate conclusion**

The number of rows in the filtered table match the number of missing values. We can conclude that the same 2174 clients are missing data on both 'days_employed' and 'total_income'.

These 2174 clients are a bit more than 10% of our 21525 clients in the whole dataset, which is a large enough portion for us to keep them (especially for conclusions about corelations of default, marital status, children, and loan purpose - where all these 2174 clients have full data). WE will first try to check whether the missing data could be due to the specific client characteristic, such as employment type or something else. Later we will fill the missing values with the most representative values.  

So maybe cases with missing data on work experience and income - whould be with a matching value in 'income_type', and 'debt' columns, therefore we will investigate this now. 

In [7]:
# Let's investigate the income type of the clients witht the missing data 
filtered_data['income_type'].value_counts() 

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

We see the clients with missing data come from different income types, and no clear reason is seen here.

In [8]:
# let's see if there is any connection between the missing data clients and default on a loan:
filtered_data['debt'].value_counts() 

0    2004
1     170
Name: debt, dtype: int64

Again, missing values exist for both defaut and non-default clients, it still seems random. Maybe the default rate of the missing values group is significantly different than the whole datadet? let's see: 

In [9]:
# Checking the debt distribution in the whole dataset
data['debt'].value_counts() 

0    19784
1     1741
Name: debt, dtype: int64

**Intermediate conclusion**

The debt distribution in the original dataset (around 8%) is similar to the debt distribution of the filtered table (the missing values group), and this further supports the counclusion that the missing work & income values are accidental and random. 

But before we conclude it, we will try to check if the missing data on work & income can be related to specific loan purpose(s):

In [10]:
# Check for relation between loan purpose and missing values
filtered_data['purpose'].value_counts() 

having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
purchase of the house for my family         71
transactions with my real estate            71
housing renovation                          70
transactions with commercial real estate    70
buy commercial real estate                  67
buying property for renting out             65
property                                    62
real estate transactions                    61
buy residential real estate                 61
housing                                     60
building a property                         59
cars                                        57
going to university                         56
to become educated                          55
second-hand car purchase                    54
buying my own


**Intermediate conclusion**

Again - it seems all loan purposes take part in the group of clients with missing work & income data, no clear reason or pattern is observed. A differnet issue we see here is that there are many different names for similar loan purposes - we will deal with this issue a bit later.

**Conclusions**

We did not find any patterns, and it seems that the missing values are not easily explained by an identifier characteristic. 

Our approach for the missing values will be later on to fill them with the most appropriate typical values (mean/median of relevant groupings), as will be explained below. Yet, when making project conclusions about income_level - results will be validated by checking the results also without the filled missing income values.  

Our next steps will be transforming data: we will address different types of issues, some of which we have detected above: duplicates, different registers, incorrect artifacts, and missing values.

## Data transformation

### Problematic values, column by column 

We will go through each column to see what issues we may have in them, starting with **education**.

In [11]:
# Let's see all values in education column to check if and what spellings will need to be fixed
data['education'].unique() 

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

It seems that changing all registers to lower key spelling will consolidate the implicit duplicates:

In [12]:
# Fix the registers if required
data['education'] = data['education'].str.lower() 

In [13]:
# Checking all the values in the column to make sure we fixed them
data['education'].unique() 

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

'Education' looks ok now. We will move now to **children** column:

In [14]:
# Let's see the distribution of values in the `children` column
data['children'].value_counts() 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

2 problems we can see: 76 cases of 20 children, and 47 cases of -1 child. Both are definately errors. The percentage of problematic data is not high (about half a percent of the dataset).

Since the number of children (and it's corelation with default on a loan) is one of the main focused purposes of this project, and since the percentage of this problematic data is so low - we will delete these 123 rows from our dataset. It could be argued that the -1 are maybe all actually 1 child, or that we could fill the wrong values with mean/median - but following the 2 reasons mentioned above - we think it will be more reliable for the purpose of this project, and still allowing to use almost all the data, to delete these rows. 

In [15]:
# fixing the data based on our decision: 
data = data[data['children'] != 20] 
data = data[data['children'] != -1]

In [16]:
# Checking the `children` column again to make sure it's all fixed
data['children'].value_counts() 

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Next, we will check the data in the **days_employed** (work experience) column.

We have noticed above 2 problems with the existing values in this column: 1. That there are negative values in this column. 2. Values of very high number (above 300000).

We will start with the negative values, let's see what is the amount of this problematic data:  

In [17]:
# count the negative values in `days_employed`, and calculate the percentage
neg_emp = data[data['days_employed'] < 0] 
neg_emp['days_employed'].count() / len(data) 

0.7386692832445566

We see the amount of problematic data here is very high (almost 74% (!) of all the clients). It could've been due to some technical error. We want to propose the most obvious reason why it could've happened: An error in the minus sign. We have no other data to suggest the values themselves (absolute values) are wrong, so we belive the correct data is probably the absolute values. In any case the variable of work experience is not a focus of this project, so we don't need to worry about reaching wrong conclusions because of uncertain values. As we can't drop these problematic rows (that do have important data about debt, family status and children) -  we will transform all 'days_employed' values to their absolute value (and round them for simplicity and convenience). For now we will keep them as float (in order to enable working with NaN values), and after we fill all NaN values later - we will change them to integers. 

In [18]:
# Round values and change them to their absolute value
data['days_employed'] = data['days_employed'].round()
data['days_employed'] = data['days_employed'].abs() 

In [19]:
# Check the result - make sure it's fixed - first we will check first 5 rows to see the change:
print(data['days_employed'][:5])

0      8438.0
1      4025.0
2      5623.0
3      4125.0
4    340266.0
Name: days_employed, dtype: float64


In [20]:
# Now we will make sure no more negative values appear:
neg_emp = data[data['days_employed'] < 0] 
neg_emp['days_employed'].count() 

0

No more negative values, but row number 4 above (340266 days, more than 932 years...) reminds us of the second problem: error of extremely high values. We will explore now this problem. As the most extreme allowed max we will take 20000 days (around 55 years of work experience, seems a safe max that any number above it is probably an error).  

In [21]:
# exploring the data with too high values for 'days_employed'
err_data = data[data['days_employed'] > 20000]
err_data['days_employed'].describe().round() 

count      3431.0
mean     365026.0
std       21087.0
min      328729.0
25%      346649.0
50%      365287.0
75%      383293.0
max      401755.0
Name: days_employed, dtype: float64

So all these values are above 328000 days (definatelly errors, almost 900 years and more!). 
Are all of them, like the first one we have detected in the sample above, retirees?

In [22]:
# Check 'income_type' values of the borrowers with huge "day_employed' values
err_data['income_type'].value_counts() 

retiree       3429
unemployed       2
Name: income_type, dtype: int64

What is the percentage of this problem (out of all borrowers data)?

In [23]:
# Percentage of the too high values
err_data['days_employed'].count() / len(data) 

0.16031212036258294

16% is quite a large part of the borrowers in the data, on which we do have data on debt, family status, children and more - so we would not want to remove these rows. But we dont have any reliable clue about the correct values for 'days_employed' for these rows. Therfore we shall replace them with NaN. Another option could have been to remove the whole column, since it is problematic in it's reliability and not needed for the questions of the current project, but for now turning them into NaN is the option we choose since it is valid and keeps as much data as possible. 

In [24]:
# replacing the too-high values in 'days_employed' with NaN, and checking it worked
import numpy as np

data.loc[data['days_employed'] > 20000, 'days_employed'] = np.nan
data['days_employed'].max()  

18389.0

Now we have no more huge (error) values in 'days_employed' column. Later, after we fill in all missing values, we will cast the velues in this column to an integer type.    

Let's now look at the **client's age** ('dob_years' column) and whether there are any issues there.

In [25]:
# Check the `dob_years` for suspicious values and their amount 
data['dob_years'].value_counts()   

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

The problematic value here is the age 0, that appears 100 times. This is only around half a percent of the dataset, but since the age is not a main focus of this research, and since these 100 rows do have important data on debt, family status and children, it feels safe enough to transform the age 0 into the mean age of the whole dataset. In any case because it is so small amount of rows - the average exact choice is not critical. 

In [26]:
# Change the problematic age with the mean age
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean() 

In [27]:
# Check the result - make sure no more ages 0 appear
(data['dob_years'] == 0).sum()  

0

But another outcome of turning the '0' values into the mean - is that the whole column might change it's dtype from int to float. Let's check if it happened here:

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       15809 non-null float64
dob_years           21402 non-null float64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        19240 non-null float64
purpose             21402 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


Yes it did. So now we will turn this column back to int type, and check it worked.

In [29]:
# convert 'dob_years' into integers and checking it worked
data['dob_years'] = data['dob_years'].astype('int') 
data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       15809 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        19240 non-null float64
purpose             21402 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Let's now look at the **client's age** ('dob_years' column) and whether there are any issues there.
Now let's check the **family_status** column. We will check what kind of values there are, and if there are issues to adress.

In [30]:
# Let's see the values for the column
data['family_status'].value_counts()  

married              12302
civil partnership     4160
unmarried             2799
divorced              1189
widow / widower        952
Name: family_status, dtype: int64

No problems are seen here. 

We will move to the **gender** column, to see what kind of values there are and wether there are problems we may need to address.

In [31]:
# Let's see the values in the column
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Again, No problems are seen here. It is optional to drop the row with the only XNA - but we don't have to do it.   

We will move now to the **income_type** column, to see what kind of values there are and wether there are problems we may need to address. 

In [32]:
data['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Here too, no problems are seen. 

let's move to the **loan purpose** column (that we have already mentioned above that will need registers' consolidation).
first let's see what are all the values in the column: 

In [33]:
# Let's see the values in the column
data['purpose'].unique() 

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

As wee can see when looking through the unique values - we can group together the loan purposes under 4 purpose groups:
1. Housing / real estate
2. Car
3. education
4. Wedding

We will now transform the **purpose** values to this simple limited amount of unique values, and then check it's changed. 


In [34]:
# replacing loan purpose to the simpler and more generalized purpose 
data = data.replace(
    ['purchase of the house', 'housing transactions', 'purchase of the house for my family', 'buy real estate',
     'buy commercial real estate','buy residential real estate', 'construction of own property', 'property',
     'building a property', 'transactions with commercial real estate', 'building a real estate', 'housing',
     'transactions with my real estate', 'purchase of my own house', 'real estate transactions', 
     'buying property for renting out', 'housing renovation'], 'housing / real estate')
data = data.replace(
    ['car purchase', 'buying a second-hand car', 'buying my own car', 'cars', 'second-hand car purchase', 'car', 'to own a car',
     'purchase of a car', 'to buy a car'], 'car')
data = data.replace(
    ['supplementary education', 'education', 'to become educated', 'getting an education', 'to get a supplementary education',
     'getting higher education', 'profile education', 'university education', 'going to university'], 'education')
data = data.replace(
    ['to have a wedding', 'having a wedding', 'wedding ceremony'], 'wedding')   

In [35]:
# Check the result - make sure it's changed
data['purpose'].unique() 

array(['housing / real estate', 'car', 'education', 'wedding'],
      dtype=object)

### Duplicates
we will check for duplicates in our data and address them/

In [36]:
# Checking duplicates
data.duplicated().sum()

408

These are totally identical rows, which is obviously an error, so they need to be dropped, and no-duplicates state validated.  

In [37]:
# Drop the duplicates
data = data.drop_duplicates().reset_index(drop
= True)

In [38]:
# Last check whether we have any duplicates
data.duplicated().sum()

0

In [39]:
# let's check the new size of the dataset that we now have after our first manipulations with it
data.shape

(20994, 12)

Before we move to filling the missing values, let's describe our **new dataset**:

We have now 20994 rows, after the following changes:
1. 408 duplicated rows were removed.
2. 123 rows with error on number of children were removed (because it is a main analysis target of this project). 
3. Problematic values were changed into:

    3.1 Column mean (age 0)
    
    3.2 Absolut value (negative number of days work experience)
    
    3.3 NaN (huge numbers of days work experience) 
    
    3.4 Consolidated into groups / same spelling (loan purpose and education, respectively) 

Total drops so far were less than 2.5% of original dataset.

The age mean change was for only half a percent of the data.

The absolut value change for number of days experience was for almost 3/4 of the data rows, and the huge values turned into NaN are another 16% of the data rows - but we will not make conclusions about this variable. 

### Working with missing values

Althogh 2 columns provide IDs for values in other columns (education_id and family_status_id), and act as simplifying dictionaries, we will prefer continue working with the full value columns for higher clarity of meaning. 

We recall the two columns having missing values are **total_income** and **days_employed**.

We will analyze which variables have high impact on each of these columns (when data exists), and choose a fitting mean/median to fill these quantitative missing values. 

####  Restoring missing values in `total_income`

We will start with addressing **total income** missing values.

In order to enable comfortable checking of  age impact, first we will create **age categories** for clients (youngest age being 19). we will create a new column with the age category. This strategy might help with calculating values for the total income.


In [40]:
# a function that calculates the age category
def age_category(age):
    if 19 <= age <= 29:
        return '19-29'
    if 30 <= age <= 39:
        return '30-39'
    if 40 <= age <= 49:
        return '40-49'
    if 50 <= age <= 59:
        return '50-59'
    if 60 <= age <= 69:
        return '60-69'
    if 70 <= age:
        return '70+'   

In [41]:
# a test if the function works
print(age_category(26))
print(age_category(30))
print(age_category(47))
print(age_category(51))
print(age_category(69))
print(age_category(70))

19-29
30-39
40-49
50-59
60-69
70+


In [42]:
# Creating new column based on the function
data['age_category'] = data['dob_years'].apply(age_category)

In [43]:
# Checking how many values for each age_cetegory in the new column, to see the distribution is reasonable 
data['age_category'].value_counts()

30-39    5566
40-49    5323
50-59    4535
19-29    3137
60-69    2265
70+       168
Name: age_category, dtype: int64

Soon we will check income correlation with these age categories, and other factors, in order to decide how to fill the missing income values. 

**Factors on which income might depend**

1. In addition to **age**, other factors that might impact income can be **education, income type (employment type), and gender**. 
2. **Loan purpose** might also have significant correlation with income, and might be significant for grouping
3. We will also need to decide whether to use **mean or median** values for replacing missing values, regardless of the charecteristic grouping that we will use. To make this decision we will look at the distribution of the factors we have identified as potentially impacting one's income.

In [44]:
#first we will round the 'total_income' values
data['total_income'] = data['total_income'].round()

For the purpose of filling / restoring the missing 'total_income' values - we will create a table that only has data that includes 'total_income' values. For this - before we 'dropna()' - we will first drop the 'days_employed' column (because we have created there more than 3000 NaN values instead of the error, and we want to use these rows, that did have 'total_income' values, for the total_income statistics). 

In [45]:
# Create a table without 'days_employed' and then - without missing values 
# and print a few of its rows to make sure it looks fine. We will not change index.
data_no_de = data.drop('days_employed', axis=1)
data_nona = data_no_de.dropna() 
data_nona.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.0,housing / real estate,40-49
1,1,36,secondary education,1,married,0,F,employee,0,17933.0,car,30-39
2,0,33,secondary education,1,married,0,M,employee,0,23342.0,housing / real estate,30-39
3,3,32,secondary education,1,married,0,M,employee,0,42821.0,education,30-39
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25379.0,wedding,50-59
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.0,housing / real estate,19-29
6,0,43,bachelor's degree,0,married,0,F,business,0,38484.0,housing / real estate,40-49
7,0,50,secondary education,1,married,0,M,employee,0,21732.0,education,50-59
8,2,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.0,wedding,30-39
9,0,41,secondary education,1,married,0,M,employee,0,23108.0,housing / real estate,40-49


We can see in this sample that row number 12 (with the NaN values as seen above) was dropped.

Now we shall check mean and median values for income - grouped by age categories: 

In [46]:
# Look at the rounded mean values for income based on your identified factors
data_nona.groupby('age_category')['total_income'].mean().round() 

age_category
19-29    25547.0
30-39    28315.0
40-49    28514.0
50-59    25808.0
60-69    23237.0
70+      20126.0
Name: total_income, dtype: float64

Before checking median, we will check if we really need to use for these calculations the "no-null-data" or we can actually use our full dataset, since both 'mean' and 'median' methods in pandas have a parameter of 'skipna' (whose defalt value is True) that calculates mean and median while skipping all the null values. Let's see that we get the same results as above when we use the full dataset:  

In [47]:
data.groupby('age_category')['total_income'].mean().round() 

age_category
19-29    25547.0
30-39    28315.0
40-49    28514.0
50-59    25808.0
60-69    23237.0
70+      20126.0
Name: total_income, dtype: float64

We can see these are the exact same results, so we can continue for now witht the 'data' df (we will return to 'data_nona' later on below).

Now we shall check median income, grouped by age category: 

In [48]:
# Look at the rounded median values for income based on age category
data.groupby('age_category')['total_income'].median().round()  

age_category
19-29    22781.0
30-39    24700.0
40-49    24770.0
50-59    22196.0
60-69    19811.0
70+      18752.0
Name: total_income, dtype: float64

Since all groups median values are notably smaller than their respective mean values - it means the income distribution is right-skewed (and not balanced). In such cases we prefer to use **median** values as a better representative for the typical case. 

But is the age category the most impactful factor? lets look at the mean & median income grouped by the other factors we have identified above:

starting with **education** 

In [49]:
# first mean income
data.groupby('education')['total_income'].mean().round()

education
bachelor's degree      33167.0
graduate degree        27960.0
primary education      21145.0
secondary education    24588.0
some college           29039.0
Name: total_income, dtype: float64

In [50]:
# and now median income
data.groupby('education')['total_income'].median().round()  

education
bachelor's degree      28086.0
graduate degree        25162.0
primary education      18742.0
secondary education    21829.0
some college           25664.0
Name: total_income, dtype: float64

**Education conclusions** 
1. Again a right-skewed distribution, making the **median** a better typical value to use for filling missing values.
2. The differences show that **education** seems more impactful on income than **age category**

Next we will check if **loan purpose** has notable corelation with **income** 

In [51]:
data.groupby('purpose')['total_income'].mean().round()  

purpose
car                      26762.0
education                26312.0
housing / real estate    27035.0
wedding                  26537.0
Name: total_income, dtype: float64

**Loan_purpose conclusion**  
Interesting to see - no notable differences in mean income, when the data is grouped by loan purpose. We will not use this grouping for filling the missing values. 

Next we will check the impact of **income type** on **income**

In [52]:
# first mean income
data.groupby('income_type')['total_income'].mean().round()

income_type
business                       32414.0
civil servant                  27319.0
employee                       25818.0
entrepreneur                   79866.0
paternity / maternity leave     8613.0
retiree                        21952.0
student                        15712.0
unemployed                     21014.0
Name: total_income, dtype: float64

In [53]:
# and now median income
data.groupby('income_type')['total_income'].median().round()

income_type
business                       27603.0
civil servant                  24067.0
employee                       22814.0
entrepreneur                   79866.0
paternity / maternity leave     8613.0
retiree                        18957.0
student                        15712.0
unemployed                     21014.0
Name: total_income, dtype: float64

**Income_type observations**
1. Usually again a right-skewed distribution (median smaller than mean), making the **median** a better typical value to use for filling missing values. The groups where median=mean probably have only 1-2 clients (we will check this point now).
2. The differences show that **income_type** seems significantly impactful on income. 

Before we move on - let's check how many clients are there in each income_type group - to better understand the numbers we got above.

In [54]:
# for the whole dataset
data['income_type'].value_counts() 

employee                       10829
business                        5000
retiree                         3715
civil servant                   1444
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

As we thought - groups with mean=median income have only 1-2 clients. Now let's see what are the income_types of the clients with the missing income values: 

In [55]:
# only among the clients with missing income values - the values we want to fill
filtered_data = data[(data['days_employed'].isnull() == True) & (data['total_income'].isnull() == True)]
filtered_data['income_type'].value_counts() 

employee         880
business         450
retiree          286
civil servant    137
entrepreneur       1
Name: income_type, dtype: int64

**Income_type conclusion**
1. Grouping by income_type is important for filing typical income when income is a missing value (with 1 exception - the 1 entrepeneur with income missing value, whom it will be a bit unreliable to fill based only on the 1 other entrepeneur, that earns a relatively high amount). 
2. **Median** income is a better typical value than mean, based on the 'income_type' grouping as well. 


**Multiple factors grouping**

Following our findings so far - we will now try to find income medians when the data is grouped_by 3 columns: education, income type, and gender (which we haven't checked yet separately but we will see here if as suspected it has notable impact on income as well)

In [56]:
# median income of groups grouped by 3 factors that seem notably impactful on income: 
data.groupby(['gender', 'income_type', 'education'])['total_income'].median().round()

gender  income_type                  education          
F       business                     bachelor's degree      30420.0
                                     primary education      21442.0
                                     secondary education    23177.0
                                     some college           27318.0
        civil servant                bachelor's degree      25255.0
                                     graduate degree        17823.0
                                     primary education      14339.0
                                     secondary education    20104.0
                                     some college           21856.0
        employee                     bachelor's degree      24603.0
                                     primary education      18963.0
                                     secondary education    19812.0
                                     some college           21588.0
        entrepreneur                 bachelor's degree     

The internal differences in median income between the different subgroups above - seem to confirm our strategy that a combination of these three factors defines most the typical income value to fill for the clients with missing data on total_income. These factors show higher income differentiation than did age_category so we will not use age here. For the entrepeneur we will use only the other 2 identifier factors: education and gender. 

**Filling the missing 'total_income' values** 

In [57]:
# filling in missing total_income values based on the medians in the multiple groupby chosen, 
# and checking how many are still null 

data['total_income'].fillna(data.groupby(['gender', 'income_type', 'education'])['total_income']
                            .transform('median'), inplace=True)
data['total_income'].isna().sum() 

1

The 1 left is the entrepeneur, so as mentioned above we will repeat the method, but this time without 'income_type' in the groupby method

In [58]:
data['total_income'].fillna(data.groupby(['gender', 'education'])['total_income']
                            .transform('median'), inplace=True)

In [59]:
# Check that it worked
data['total_income'].isna().sum() 

0

No more missing values for 'total_income'. Let's check that our method worked and didn't fill the same median to all the missing cases:

In [60]:
# check sample of 15 highest_count total_income values - which logically should be all different medians
# that were created by the 'transform('median')' method with 'groupby' above: 
data['total_income'].value_counts().head(15) 

19812.5    382
25010.0    242
18025.0    186
23177.0    157
24603.0    132
30420.5    116
29133.0    100
30893.5     75
37400.0     59
20104.0     54
25255.0     46
19927.0     45
22769.5     41
21588.0     23
26991.5     16
Name: total_income, dtype: int64

As last check after finishing with `total_income`, we will check that the total number of values in this column matches the number of values in other ones - which is in the clean dataset, as we recall, 20994. 

In [61]:
# Checking the number of entries in the columns
data['total_income'].count() 

20994

All ok here. Let's move to the second column with missing values: 'days_employed'. 

####  Restoring missing values in `days_employed`

We will conduct here a research similar (but shorter) to the one we've done when restoring the data in 'total_income' column. Main factors that logically impact 'days_employed' work experience - might be: age category, income_type (employment type), education.  

In [62]:
# Distribution of `days_employed` means based on age-category
data.groupby('age_category')['days_employed'].mean().round() 

age_category
19-29    1209.0
30-39    2026.0
40-49    2726.0
50-59    3267.0
60-69    3829.0
70+      4227.0
Name: days_employed, dtype: float64

In [63]:
# Distribution of `days_employed` medians based on age_category 
data.groupby('age_category')['days_employed'].median().round()

age_category
19-29     997.0
30-39    1593.0
40-49    2012.0
50-59    2266.0
60-69    2663.0
70+      2680.0
Name: days_employed, dtype: float64

**Age_category conclusions**
1. Age_category is definatelly a major identifier factor for days_employed, differences between groups being so significant. Due to it's large (expected) impact on days_employed - it seems that 1 more identifier should be enough.  
2. **Median** seems more typical, due to the big differences between mean and median Next we will check in each group.

Next we will check 'days_employed' grouped by 'income_type':

In [64]:
# Distribution of `days_employed` means based on income_type
data.groupby('income_type')['days_employed'].mean().round() 

income_type
business                       2118.0
civil servant                  3404.0
employee                       2324.0
entrepreneur                    521.0
paternity / maternity leave    3297.0
retiree                           NaN
student                         579.0
unemployed                        NaN
Name: days_employed, dtype: float64

In [65]:
# Distribution of `days_employed` medians based on age-category
data.groupby('income_type')['days_employed'].median().round() 

income_type
business                       1556.0
civil servant                  2689.0
employee                       1572.0
entrepreneur                    521.0
paternity / maternity leave    3297.0
retiree                           NaN
student                         579.0
unemployed                        NaN
Name: days_employed, dtype: float64

**Income_type conclusions**
1. Income_type groups do show significant differences in 'days_employed' means and medians, and can be one of the important predictors of 'days_employed' - except the cases of retiree (3429 rows, see above) and unemployed (2 rows), which had error huge 'days_employed' values, and we have changed into NaN. 
2. **Median** more typical than mean, due to big differences with mean in some of the groups (in most of the groups that are relevant for the clients with the missing values). 

**Multiple factors grouping** 

Before we combine the impact of income_type, we will check 'days_employed' grouped by both 'age_category' and 'education' (using median as we see that the income distribution is skewed in most cases) 

In [66]:
data.groupby(['age_category', 'education'])['days_employed'].median().round()

age_category  education          
19-29         bachelor's degree      1006.0
              primary education      1030.0
              secondary education    1014.0
              some college            886.0
30-39         bachelor's degree      1598.0
              graduate degree        4160.0
              primary education      1090.0
              secondary education    1626.0
              some college           1222.0
40-49         bachelor's degree      2020.0
              graduate degree         409.0
              primary education      1040.0
              secondary education    2022.0
              some college           1996.0
50-59         bachelor's degree      2376.0
              graduate degree        5352.0
              primary education      2266.0
              secondary education    2244.0
              some college           1898.0
60-69         bachelor's degree      3000.0
              graduate degree           NaN
              primary education      4016.

**Conclusions about 'education' and 'age_category' together**

Even inside quite a few of the age groups - education matters to 'days_employed' median. This supports our strategy of filling the missing values with multiple factors groupby, so each client with missing value will be asigned a median that matches his charecteristics in a few influencial identifiers. We will need to solve the "missing medians" for all cases were the error huge values of all retirees (that we have changed into NaN) left us with little reliable data about the 'days_employed' that could be typical to some of the borrowers. This limitation of accuracy is less critical for this project since we are not going to answer research questions that involve 'days_employed' (work experience).  

**Filling the missing values in 'days_employed'**

Following all of the above - we will now fill the missing values in 'days_employed' with the median based on 3 factors: age_category, education, and income_type. We will use the same methods (transform and groupby) as we did it above with 'total_income'

In [67]:
# filling in missing 'days_employed' values based on the medians in the multiple groupby chosen, 
# and checking how many are still null 

data['days_employed'].fillna(data.groupby(['age_category', 'education', 'income_type'])['days_employed']
                            .transform('median'), inplace=True)
data['days_employed'].isna().sum() 

3720

These are mainly the retirees, as discussed above. Since groupby 'income_type' cannot help here - we will first fill the missing values using median based on groupby of only the other 2 variables - 'age_category' and 'education'. Then we will again check how many are still null.

In [68]:
data['days_employed'].fillna(data.groupby(['age_category', 'education'])['days_employed']
                            .transform('median'), inplace=True)
data['days_employed'].isna().sum() 

13

For the last 13 cases - our solution will be to use the median 'days_employed' groupby (only) the age_category, which seems like the highest impact factor on days_employed (both by the evidence above and logically).  

In [69]:
# filling in last 13 missing 'days_employed' values based on the medians of age_category alone, 
# and checking how many are still null 

data['days_employed'].fillna(data.groupby('age_category')['days_employed']
                            .transform('median'), inplace=True)
data['days_employed'].isna().sum() 

0

No more missing values for 'days_employed'. Let's check that our method worked and didn't fill the same median to all the missing cases:

In [70]:
# check sample of 15 highest_count days_employed values - which logically should be all different medians
# that were created by the 'transform('median')' method with 'groupby' above: 
data['days_employed'].value_counts().head(15) 

2462.5    1439
2119.0    1420
1913.5     307
3059.5     263
2490.0     256
1571.0     244
1899.0     131
1016.0     119
1647.5      84
1898.0      78
1877.0      75
1497.0      74
1428.0      73
1855.5      62
4016.0      61
Name: days_employed, dtype: int64

As last check after finishing with 'days_employed', we will check that the total number of values in this column matches the number of values in other ones - which is in the clean dataset, as we recall, 20994.

In [71]:
data['days_employed'].count() 

20994

In [72]:
# Checking the entries in all columns - making sure we fixed all missing values
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994 entries, 0 to 20993
Data columns (total 13 columns):
children            20994 non-null int64
days_employed       20994 non-null float64
dob_years           20994 non-null int64
education           20994 non-null object
education_id        20994 non-null int64
family_status       20994 non-null object
family_status_id    20994 non-null int64
gender              20994 non-null object
income_type         20994 non-null object
debt                20994 non-null int64
total_income        20994 non-null float64
purpose             20994 non-null object
age_category        20994 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Now that all 'days_employed' null values are filled - it's time to convert the values from float to integer:

In [73]:
# convert 'days_employed' into integers and checking it worked
data['days_employed'] = data['days_employed'].astype('int') 
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994 entries, 0 to 20993
Data columns (total 13 columns):
children            20994 non-null int64
days_employed       20994 non-null int64
dob_years           20994 non-null int64
education           20994 non-null object
education_id        20994 non-null int64
family_status       20994 non-null object
family_status_id    20994 non-null int64
gender              20994 non-null object
income_type         20994 non-null object
debt                20994 non-null int64
total_income        20994 non-null float64
purpose             20994 non-null object
age_category        20994 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


We can see 'days_employed' is now int type. 

## Categorization of data (continued) 

Before we can see which further categorization we might need, let's look again at the 4 questions we need to answer:
1. Is there a connection between having kids and repaying a loan on time?
2. Is there a connection between marital status and repaying a loan on time?
3. Is there a connection between income level and repaying a loan on time?
4. How do different loan purposes affect on-time loan repayment?

We have already created categorization for age_category and simplified the loan purposes into 4 main purposes.

With number of children and marital status (the main focus of this project) we will work with the values as they are, since there are not too many unique values in each of these factors, and clarity and accuracy will be better without creating new categories there.

We do need to create now categories for 'income_level', to enable answering the third question.  

In [74]:
# Getting summary statistics for 'total_column' column
data['total_income'].describe().round() 


count     20994.0
mean      26555.0
std       15873.0
min        3307.0
25%       17066.0
50%       23177.0
75%       31667.0
max      362497.0
Name: total_income, dtype: float64

Following the above statistics, we will create 4 'income level' categories, close to the 4 quartiles: 

In [75]:
# Creating function for categorizing into different numerical groups based on ranges
def income_level(income):
    if income <= 17000:
        return 'up to 17K'
    if 17000 < income <= 23000:
        return '17K-23K'
    if 23000 < income <= 32000:
        return '23K-32K'
    if 32000 < income:
        return '32K+'

In [76]:
# Creating column with categories
data['income_level'] = data['total_income'].apply(income_level)

In [77]:
# Count each categories values to see the distribution
data['income_level'].value_counts() 

23K-32K      5666
up to 17K    5201
32K+         5112
17K-23K      5015
Name: income_level, dtype: int64

## Answering the questions 

### Is there a correlation between having children and paying back on time?

In [78]:
# first let's see what is the overall default rate in the whole dataset - as benchmark
# since 'debt' gets a '0' for paying on time and '1' for not paying back on time - it's mean value represents the defoult rate 
data['debt'].mean()

0.08240449652281605

So the general default rate for loans in the whole dataset is 8.24%. All following reults for the different factors will be compared to this result. 

In [79]:
# Calculating default-rate based on the number of children
data.groupby('children')['debt'].mean() 

children
0    0.076734
1    0.093474
2    0.095145
3    0.082067
4    0.100000
5    0.000000
Name: debt, dtype: float64

In [80]:
# Before we draw the conclusions - lets' look again at the amount of clients in each group:
data['children'].value_counts()

0    13827
1     4750
2     2039
3      329
4       40
5        9
Name: children, dtype: int64

**Conclusions**

1. Clients with 3 kids have a default rate similar to the overall general rate (8.21%). 

2. Clients without kids (the biggest group) have the lowest default rate (7.67%).

3. Clients with 1-2 kids (the big majority of all clients with kids) have significantly higher default rate (9.35%, 9.51%).

4. Clients with 4 kids have the highest default rate (10%) but this is a much smaller group in the dataset, so this result should be taken as not statistically solid.
(The data about clients with 5 children is too small to be based on)

5. **Overall** - having children is correlated with higher default rates on loans than clients without children.

### Is there a correlation between family status and paying back on time?

In [81]:
# Calculating default-rate based on family status
data.groupby('family_status')['debt'].mean() 

family_status
civil partnership    0.093765
divorced             0.070767
married              0.077109
unmarried            0.098556
widow / widower      0.067380
Name: debt, dtype: float64

In [82]:
# Before we draw the conclusions - lets' look at the amount of clients in each group:
data['family_status'].value_counts()

married              11996
civil partnership     4106
unmarried             2770
divorced              1187
widow / widower        935
Name: family_status, dtype: int64

**Conclusions**

1. Widow / widower clients have the lowest default rate (6.74%).  
2. Divorced clients have slightly higher default rate (7.08%).
3. Married clients (the biggest group, more than half of the dataset) have higher default rate (7.71%), but still lower than the general default rate of 8.24%.
4. Civil partnership clients have significantly higher default rate (9.38%)
5. Unmarried clients have the highest default rate (9.86%)
6. **Overall** - marital status of widow/er, divorced and married are related to lower default rates, while civil partnership and unmarried clients have significantly higher default rates. 

### Family status & number of children  - Shared relation to paying back on time

For credit score purposes it will be beneficial to look at both factors - marital status and number of children - in their shared relation to paying back on time. For this we will create and present a pivot table, and clarify it's conclusions.

In [83]:
# a pivot table for default rate based on family status and number of children 
data_pivot = data.pivot_table(index=['family_status'], columns='children',
values='debt', aggfunc='mean' , margins=True)
data_pivot 

children,0,1,2,3,4,5,All
family_status,,,,,,,
civil partnership,0.084564,0.119192,0.087719,0.142857,0.000000,0.0,0.093765
divorced,0.070332,0.067308,0.086420,0.090909,0.000000,NaN,0.070767
married,0.070702,0.084189,0.095332,0.068548,0.107143,0.0,0.077109
unmarried,0.093918,0.115813,0.120000,0.125000,0.500000,NaN,0.098556
widow / widower,0.063779,0.090909,0.150000,0.000000,0.000000,NaN,0.067380
All,0.076734,0.093474,0.095145,0.082067,0.100000,0.0,0.082404


In [84]:
# Before we draw the conclusions - lets' look at the amount of clients in each group:
data_pivot2 = data.pivot_table(index=['family_status'], columns='children',
values='debt', aggfunc='count', margins=True)
data_pivot2

children,0,1,2,3,4,5,All
family_status,,,,,,,
civil partnership,2708.0,990.0,342.0,56.0,8.0,2.0,4106
divorced,782.0,312.0,81.0,11.0,1.0,NaN,1187
married,7270.0,2922.0,1521.0,248.0,28.0,7.0,11996
unmarried,2236.0,449.0,75.0,8.0,2.0,NaN,2770
widow / widower,831.0,77.0,20.0,6.0,1.0,NaN,935
All,13827.0,4750.0,2039.0,329.0,40.0,9.0,20994


**Conclusions**

1. The upper table provides the shared default rate - for both factors (family_status & number of children). Example results can be the high default rate of civil partnership with 1 child (11.92%) on one hand, and the low default rate of widow/er without children (6.38%) on the other hand. This shows that the dual factor can double(!) the default rate. 
2. The statistical representativeness of each cell is dependant on the size of the sample (seen in the lower table), and therefore is limited in some cases.
3. **Overall** it is recomended to consider both family-children factors when defining a borrower's credit score. 

### Is there a correlation between income level and paying back on time?

In [85]:
# Calculating default-rate based on income level 
data.groupby('income_level')['debt'].mean() 

income_level
17K-23K      0.091924
23K-32K      0.088422
32K+         0.070227
up to 17K    0.078639
Name: debt, dtype: float64

**Conclusions**

1. Highest income_level borrowers have the lowest default rate (7.02%).
2. lowest income_level borrowers have higher default rate (7.86%), but still lower than the overall default rate (8.24%). 
3. 2nd and 3rd income quartiles have the highest default rates (9.19%, 8.84%)

**Reliability check**

Since we have filled more than 2000 missing total_income values - it is important to check that these filled values havn't changed dramatically our income-impact conclusions. Therfore we will redo the defoult-rate based on income_level for the data without the initial missing income values (created above as 'data_nona'). 

In [88]:
# first creating the income_level column for the data_nona (without the originally missing income values)
data_nona['income_level'] = data_nona['total_income'].apply(income_level)

# Calculating default-rate based on income level (without the originally missing income values)
data_nona.groupby('income_level')['debt'].mean() 

income_level
17K-23K      0.088972
23K-32K      0.088012
32K+         0.070677
up to 17K    0.078745
Name: debt, dtype: float64

We can see the differences from above are not big, and our conclusions about income_level vs. default rate are validated. 

### How does credit purpose affect the default rate?

In [89]:
# Check the percentages for default rate for each credit purpose and analyze them
data.groupby('purpose')['debt'].mean() 

purpose
car                      0.093993
education                0.093679
housing / real estate    0.074085
wedding                  0.079739
Name: debt, dtype: float64

**Conclusion**

1. Loans for **housing / real estate** have the lowest default rate (7.41%).
2. Loans for **wedding** have a bit higher default rate (7.97%), but still lower than the overall rate (8.24%).
3. Loans for **car or education** have both significantly higher default rate (9.4%, 9.37%)



## General Conclusion 

### Data preprocessing 
1. Original dataset of 21525 rows (loans) was analyzed, and 2174 rows with missing values (on income and work experience) were detected. No patterns were found for the missing values.

2. 408 duplicated rows were removed.

3. 123 rows with error on number of children were removed (because it is a main analysis target of this project).

4. The new dataset includes 20994 reliable rows.

5. Problematic values were changed to:

    5.1. Column mean (age 0)
    
    5.2. Absolute value (negative number of days work experience)
    
    5.3 NaN (huge numbers of days work experience) 
    
    5.4 Consolidated into groups / same spelling (loan purpose and education, respectively) 
    
6. Total_income missing values were filled using the median income, grouped by gender, income_type, and education.

7. Days_employed missing values were filled using the median income, grouped by age_category, education, and income_type. 
       
8. Percentage of changes:

    8.1. Total drops were less than 2.5% of original dataset.
    
    8.2. The age mean change was for only half a percent of the data.
    
    8.3. The absolut value change for number of days experience was for almost 3/4 of the data rows, and the huge values turned into NaN are another 16% of the data rows - but no project conclusions are based on this variable. 
    
    8.4. Filled missing values (total_income & days_employed) are for around 10% of the rows, but reliability test for the income results includes also a caculation without the filled missing values. 
    
    
9. Categorization was created for age_category, loan_purpose, and income_level.

### Answering the Analysis Questions

**General Conclusion**: All 4 factors (number of children, marital status, income level, loan purpose) should be considered when defining credit score for a potential borrower, since differences in all 4 factors are correlated with differences in loan default rate.  

1. **Is there a connection between having kids and repaying a loan on time?**

    1. Clients with 3 kids have a default rate similar to the overall general rate (8.21%). 
    2. Clients without kids (the biggest group) have the lowest default rate (7.67%).
    3. Clients with 1-2 kids (the big majority of all clients with kids) have significantly higher default rate (9.35%, 9.51%).
    4. Clients with 4 kids have the highest default rate (10%) but this is a much smaller group in the dataset, so this result should be taken as not statistically solid.
(The data about clients with 5 children is too small to be based on)
    5. **Overall** - having children is correlated with higher default rates on loans than clients without children.


2. **Is there a connection between marital status and repaying a loan on time?**

    1. Widow / widower clients have the lowest default rate (6.74%).  
    2. Divorced clients have slightly higher default rate (7.08%).
    3. Married clients (the biggest group, more than half of the dataset) have higher default rate (7.71%), but still lower than the general default rate of 8.24%.
    4. Civil partnership clients have significantly higher default rate (9.38%)
    5. Unmarried clients have the highest default rate (9.86%)
    6. **Overall** - marital status of widow/er, divorced and married are related to lower default rates, while civil partnership and unmarried clients have significantly higher default rates. 
    7. A pivot table is provided above, that calculates and provides the default rate, based both on marital status and on number of children. 


3. **Is there a connection between income level and repaying a loan on time?**

    1. Highest income_level borrowers have the lowest default rate (7.02%).
    2. lowest income_level borrowers have higher default rate (7.86%), but still lower than the overall default rate (8.24%). 
    3. 2nd and 3rd income quartiles have the highest default rates (9.19%, 8.84%)
    4. These conclusions are validated when calculated also without the filled missing income values. 


4. **How do different loan purposes affect on-time loan repayment?**

    1. Loans for **housing / real estate** have the lowest default rate (7.41%).
    2. Loans for **wedding** have a bit higher default rate (7.97%), but still lower than the overall rate (8.24%).
    3. Loans for **car or education** have both significantly higher default rate (9.4%, 9.37%)